In [1]:
##############
#Dependencies#
##############
%matplotlib notebook
import datetime
import pandas
import numpy as np
import folium
import os
import time
from selenium import webdriver




In [2]:
route = ['S567','S583','S568','S582','S569','S570','S581',
'S571','S579','S572','S578','S573',"S577","S587"]
station_df = pandas.read_csv('data/station_config.csv')
route_df= pandas.read_csv('data/route_summary.csv')
station_df = station_df.set_index("Station")
route_df = route_df.set_index("Station")
route_df= route_df.replace("UNKNOWN",0)
route_df= route_df.fillna(0)
route_df.head()

,Station.1,Time,Occupancy,Speed,Flow
Station,,,,,
S567,0.0,Wed Oct 03 01:16:59 CDT 2018,0.31,54,0
S567,0.0,Wed Oct 03 01:16:59 CDT 2018,0.31,54,0
S567,0.0,Sun Oct 07 00:22:29 CDT 2018,3.83,62,560
S583,0.0,Wed Oct 03 01:16:59 CDT 2018,0.24,55,0
S583,0.0,Wed Oct 03 01:16:59 CDT 2018,0.24,55,0


In [3]:

Results = {}


locations = []

for station in route:
        Occupancy =  float(route_df.loc[station,'Occupancy'].tail(1)[0])
        Speed = int(route_df.loc[station,'Speed'].tail(1)[0])

        Max_Occupancy = route_df.loc[station,'Occupancy'].astype(dtype="float")
        Max_Occupancy = max(Max_Occupancy.values)
        Max_Speed = route_df.loc[station,'Speed'].astype(dtype="int")
        Max_Speed = max(Max_Speed.values)
        
        lat, lon = station_df.loc[station, ['Lat', 'Lng']]
        locations.append((lat, lon))
    
        Results[station] = {
            "Current Speed" : Speed,
            "Current Occupancy" : Occupancy,
            "Max Speed" : Max_Speed,
            "Max Occupancy" : Max_Occupancy,
            "% of Max Speed" : round(Speed / Max_Speed * 100, 2),
            "% of Max Occupancy" : round(Occupancy / Max_Occupancy * 100, 2) ,
            "Lat":lat, 
            "Lon":lon}

In [4]:
Results = pandas.DataFrame(Results).T
Results.head()
Results

,% of Max Occupancy,% of Max Speed,Current Occupancy,Current Speed,Lat,Lon,Max Occupancy,Max Speed
S567,52.31,81.61,4.31,71.0,44.97447,-93.24870,8.24,87.0
S583,88.75,65.93,6.47,60.0,44.98175,-93.24397,7.29,91.0
S568,24.73,64.95,2.06,63.0,44.98399,-93.24197,8.33,97.0
S582,40.98,77.36,2.93,82.0,44.99115,-93.23659,7.15,106.0
S569,92.33,67.67,3.85,90.0,44.99116,-93.23629,4.17,133.0
S570,63.56,70.18,3.07,80.0,44.99644,-93.23823,4.83,114.0
S581,53.14,85.37,4.15,70.0,44.99646,-93.23853,7.81,82.0
S571,28.70,68.09,6.09,64.0,45.00199,-93.23465,21.22,94.0
S579,43.33,86.59,3.28,71.0,45.00091,-93.22346,7.57,82.0
S572,45.27,82.35,4.69,70.0,45.00070,-93.22349,10.36,85.0


In [5]:
folium_map = folium.Map(locations[5],
                        zoom_start=13,
                        tiles="CartoDB positron")
for index, row in Results.iterrows():

    
    folium.CircleMarker(location=(row["Lat"],
                                  row["Lon"]),
                        radius=1,
                        
                        fill=True).add_to(folium_map)

In [6]:
Features = []
Last_Sensor = []
for index, row in Results.iterrows():
    Current_Sensor = (row['Lat'], row['Lon'])
    if Last_Sensor == [] :
        Last_Sensor = (row['Lat'], row['Lon'])
    else:
    
        folium.PolyLine([Current_Sensor,Last_Sensor],
         weight=row['Current Occupancy'] * 2.5,
                        popup=f"Speed:{row['Current Speed']} Occupancy: {row['Current Occupancy']}").add_to(folium_map)

        Last_Sensor = (row['Lat'], row['Lon'])





In [7]:
folium_map


In [10]:

print(f'Map saved at results/maps/routemap{datetime.datetime.now().strftime("%b%d_%H%M%S")}')

Map saved at results/maps/routemapOct07_095259


In [14]:
delay=10
fn=f'results/maps/routemap{datetime.datetime.now().strftime("%b%d_%H%M%S")}'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
folium_map.save(fn)

browser = webdriver.Firefox()
browser.get(tmpurl)
#Give the map tiles some time to load
time.sleep(delay)
browser.save_screenshot(f'results/maps/routemap{datetime.datetime.now().strftime("%b%d_%H%M%S")}.png')
browser.quit()